# Using napari to process a large dataset (brain imaging)

In this notebook, we will use napari and scikit-image to process a large brain imaging dataset and detect the edges of the brain images.

Since [napari](https://napari.org) is a GUI application, and we are using a docker image (`jupyter-server`) that doesn't provide a desktop server on our MMCloud instance, we need to set up some environment variables so that napari can run in headless mode.

In [ ]:
%set_env QT_QPA_PLATFORM=offscreen
%set_env XDG_RUNTIME_DIR=/mnt/napari

Next, we will install the Python dependencies we will use for our operations:

In [ ]:
!pip install napari[all] tornado==6.1 dask-image napari-compressed-labels-io -q

## Downloading files

The dataset we are using here is a high-resolution human brain imaging dataset. This dataset was originally described in [Edlow, B.L., Mareyam, A., Horn, A. _et al._ 7 Tesla MRI of the ex vivo human brain at 100 micron resolution. Sci Data 6, 244 (2019)](https://doi.org/10.1038/s41597-019-0254-8). We will download this dataset as a zipfile containing a set of TIFF files, and extract it to our desired location.

Note that `volume` points to the Volume we set up when creating our MMCloud Job for this notebook.

In [ ]:
volume = "/mnt/napari"
url = "https://s3.amazonaws.com/openneuro.org/ds002179/derivatives/sub-EXC004/synthesized_FLASH25_100um_tiff_images/Synthesized_FLASH25_100um_TIFF_Axial_Images.zip?versionId=6L9rb8Lv1whb1eXQVTgnYh1bSSwc_5Hj"
filename = f"{volume}/Synthesized_FLASH25_100um_TIFF_Axial_Images.zip"

In [ ]:
import requests, zipfile
r = requests.get(url, allow_redirects=True)
open(filename, 'wb').write(r.content)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(f"{volume}/.")

The TIFF files are now available for our analysis.

## Reading image files with napari

We will use [dask-image](https://image.dask.org/en/latest/) to load up our TIFF files into memory.

In [ ]:
import napari
import numpy as np
from dask_image.imread import imread

images = imread(f'{volume}/SYNTHESIZED_TIFF_Images_Raw/Synthesized_FLASH25_100um_TIFF_Axial_Images/Synthesized_FLASH25_Axial_*.tiff')

Now, we will set up our napari `Viewer` (even though it won't be shown since we are running in headless mode) and add our images to the viewer.

In [ ]:
viewer = napari.Viewer(show=True)
viewer.add_image(images, name='brain')

## Detecting the edges of the brain images

The brain images contained in this dataset form a stack of image arrays, showing different slices of the brain being processed.

We will look at each of these slices and try to detect the edges of each of these images using the [Canny edge detector from scikit-image](https://scikit-image.org/docs/stable/auto_examples/edges/plot_canny.html).

In [ ]:
from skimage import img_as_ubyte
from skimage.feature import canny
from dask.array import from_array, stack

res = []
for i in range(images.shape[0]):
    res.append(img_as_ubyte(from_array(canny(images[i], sigma=1, low_threshold=25, high_threshold=50))))
border = stack(res)

Note that both `images` and `border` are dask arrays: 

In [ ]:
images

In [ ]:
border

Finally, we can add this image to napari highlighting the edges of each of the brain image slices:

In [ ]:
viewer.add_labels(border, name="border-labels")

## Using a napari plugin to save layer data

The [napari-compressed-labels-io](https://www.napari-hub.org/plugins/napari-compressed-labels-io) plugin for napari allows us to save the current layers data as an zarr file, including the layer metadata. We will create this file and download it for future analysis.

In [ ]:
from napari_compressed_labels_io import labels_to_zarr

In [ ]:
viewer.layers

In [ ]:
edges = viewer.layers[1]

In [ ]:
edges.metadata

In [ ]:
labels_to_zarr(f"{volume}/edges.zarr", np.asarray(edges.data), edges.metadata)

To make it easier to download these files, we will add them to a zip archive first.

In [ ]:
import glob
with zipfile.ZipFile('edges.zip', 'w') as zip_ref:
    for file in glob.glob(f"{volume}/edges.zarr/*"):
        zip_ref.write(file)

Our file "edges.zip" is now ready to be downloaded from the Jupyter interface.

The labels layer we just saved can then be recovered by using

```python
border = get_zarr_labels("edges.zarr")("edges.zarr")
viewer.add_labels(border[0][0])
```